#### Library prepararion

In [ ]:
#Instalation of libraries
!pip install wandb
!pip install transformers datasets accelerate peft
!pip install auto-gptq
!pip install optimum
!pip install trl
!pip install torch
!pip install peft transformers accelerate
!pip install transformers accelerate auto-gptq
!pip uninstall bitsandbytes -y
!pip install transformers datasets peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

#### Training part

In [ ]:
import pandas as pd
import numpy as np
import re
import os


import transformers
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments,DataCollatorForLanguageModeling,AutoTokenizer,BitsAndBytesConfig,AutoModelForCausalLM

from transformers import GPTQConfig
print(transformers.__version__)
print(transformers.TrainingArguments.__module__)
import wandb

wandb.login()
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

#It looks for dataset in my google drive folder
project1_json = "/content/drive/MyDrive/project-42-at-2025-03-27-13-14-727622cd.json"
project2_json = "/content/drive/MyDrive/project-60-at-2025-03-27-13-20-fac55b6a.json"

project1_path = "./HistoryNer/NER"
project2_path = "./HistoryNer/NER_02"

server_path = "/content/drive/MyDrive/HistoryNer/NER"
server_prefix = "/content/drive/MyDrive/HistoryNer"

prefix = "https://label-studio.semant.cz/data/local-files/?d=historical_ner"
my_prefix = "./HistoryNer"
edit_prefix = "./EHistoryNer"

#File loading function
def load_file(file_path):
    if os.path.exists(file_path):
        try:
            with open(file_path, 'r') as file:
                return file.read()
        except Exception as e:
            print(f"Error reading file {file_path}: {e}")
            return None
    else:
        return None

df = pd.read_json(project1_json)
df_edit = df[['data']]
print(df.columns)

#Opem files to ['file_contents']
df_edit['data'] = df_edit['data'].apply(lambda x: x['text'])
df_edit['file_path'] = df_edit['data'].apply(lambda x: server_prefix+re.sub(f"^{re.escape(prefix)}", "", x))
df_edit['file_contents'] = df_edit['file_path'].apply(load_file)

#Remove weird annotations for now
df_edit['annotations'] = df[['annotations']]
df_edit['annotation_count'] = df_edit['annotations'].apply(lambda x: len(x))
df_edit = df_edit[df_edit['annotation_count']==1]
df_edit['annotations'] = df_edit['annotations'].apply(lambda x: x[0])
df_edit['annotations'] = df_edit['annotations'].apply(lambda x: x['result'])

#filter by language
df_edit['language'] =  df_edit['annotations'].apply(lambda x: [annotation['value']['choices'][0] for annotation in x if annotation['type'] == 'choices'])
df_edit['language_count'] = df_edit['language'].apply(lambda x: len(x))
df_edit = df_edit[df_edit['language_count']==1]
df_edit['language'] = df_edit['language'].apply(lambda x: x[0])
df_edit = df_edit[df_edit['language']=='czech']
df_edit = df_edit.drop('language_count', axis=1)

df_edit['annotations'] =  df_edit['annotations'].apply(lambda x: [annotation['value'] for annotation in x if annotation['type'] == 'labels'])
df_edit['annotations_count'] = df_edit['annotations'].apply(lambda x: len(x))
df_edit = df_edit[df_edit['annotations_count']>0]
df_edit = df_edit.drop('annotations_count', axis=1)

print(df_edit['annotations'].head())

#edit text to xml_formatting
def to_xml_formatting(text,annotations):
    begins = []
    ends = []
    for annot in annotations:
        begins.append((annot['start'],annot['labels'][0]))
        ends.append(annot['end'])
    ends.sort()
    begins = sorted(begins, key=lambda x: x[0])
    while(len(ends)!=0 or len(begins)!=0):
        next_end = ends[len(ends)-1] if len(ends)-1 >= 0 else -1
        next_begin = begins[len(begins)-1][0] if len(begins)-1 >= 0 else -1
        if next_end > next_begin:
            insert = "</ne>"
            text = text[:next_end] + insert + text[next_end:]
            ends.pop()
        else:
            insert = f"<ne type=\"{begins[len(begins)-1][1]}>\""
            text = text[:next_begin] + insert + text[next_begin:]
            begins.pop()
    return text

df_edit['annotated_text'] = df_edit.apply(lambda row: to_xml_formatting(row['file_contents'], row['annotations']), axis=1)

#Create new paths
df_edit['edit_file_path'] = df_edit['data'].apply(lambda x: edit_prefix+re.sub(f"^{re.escape(prefix)}", "", x))

#Save to new files
def save_to_file(file_path,text):
    with open(file_path, "w+") as f:
        f.write(text)

def debug_print_df_head(df_edit):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)
    print(df_edit.head(5))
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.max_colwidth')

def remove_the_path(string):
    return re.sub("^.*\/","",string)
def get_anotator(string):
    parts = re.split('\/',string)
    return parts[len(parts)-2]

#Split dataset by anotator
df_edit['file_name'] = df_edit['file_path'].apply(lambda x: remove_the_path(x))
df_edit['anotator'] = df_edit['file_path'].apply(lambda x: get_anotator(x))
df_edit['count'] = df_edit.groupby('file_name')['file_name'].transform('count')
df_edit['annot_c'] = df_edit.groupby('anotator')['anotator'].transform('count')

df_anot1 = df_edit[df_edit['anotator']=='1']
df_anot2 = df_edit[df_edit['anotator']=='2']
df_anot3 = df_edit[df_edit['anotator']=='3']
df_anot4 = df_edit[df_edit['anotator']=='4']
df_anot5 = df_edit[df_edit['anotator']=='5']

def split_dataset(df_edit:pd.DataFrame):
    duplicates = df_edit[df_edit.duplicated('file_name', keep=False)]
    singles = df_edit[~df_edit.duplicated('file_name', keep=False)]
    duplicates_len = duplicates.shape[0]
    if duplicates_len>int(len(df_edit)*(1 - 0.85)):
        return singles,duplicates
    move = int(len(df_edit)*(1 - 0.85) - duplicates_len)
    part1 = df_edit.iloc[:move]
    part2 = df_edit.iloc[move:]
    test = pd.concat([duplicates, part1], ignore_index=True)
    return test,part2

df_anot1_test,df_anot1_train = split_dataset(df_anot1)
df_anot2_test,df_anot2_train = split_dataset(df_anot2)
df_anot3_test,df_anot3_train = split_dataset(df_anot3)
df_anot4_test,df_anot4_train = split_dataset(df_anot4)
df_anot5_test,df_anot5_train = split_dataset(df_anot5)

df_train_f = pd.concat([df_anot1_train, df_anot2_train], ignore_index=True)
df_train_f = pd.concat([df_train_f, df_anot3_train], ignore_index=True)
df_train_f = pd.concat([df_train_f, df_anot4_train], ignore_index=True)
df_train_f = pd.concat([df_train_f, df_anot5_train], ignore_index=True)


#Initialize statistics tracking
wandb.init(
    project="KNN",
    name="gpt2",
    config={
        "epochs": 100,
        "batch_size": 4,
        "learning_rate": 0.001
    }
)

# Load model and tokenizer
#TinyLlama model
'''
model_name = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # Auto placement on GPUs
    quantization_config= GPTQConfig(bits=4, disable_exllama=True),
    offload_folder="offload"
)
device = torch.device('cuda:0') # Remember to use the correct device here
'''

#GPT2 model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate)

# Add padding token
tokenizer.pad_token = tokenizer.eos_token

#Tokenize lines
def tokenize(text_in,text_out):
    full_text = f"Input: {text_in}\nOutput: {text_out}"
    return tokenizer(full_text, truncation=True, padding='max_length', max_length=512)

df_train_f['tokenized']=df_train_f.apply(lambda row: tokenize(row['file_contents'], row['annotated_text']), axis=1)
df_anot1_test['tokenized']=df_anot1_test.apply(lambda row: tokenize(row['file_contents'], row['annotated_text']), axis=1)

training_args = transformers.TrainingArguments(
    output_dir="./gpt2-finetuned",
    per_device_train_batch_size=wandb.config.batch_size,
    num_train_epochs=wandb.config.epochs,
    learning_rate=wandb.config.learning_rate,
    logging_dir="./logs",
    report_to="wandb",  # this enables wandb logging
    logging_steps=50,
    save_strategy="no" #Dont save checkpoints
)

data_collator = transformers.DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

#Define trainer
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=df_train_f['tokenized'],
    data_collator=data_collator,
    eval_dataset=df_anot1_test['tokenized'],
    tokenizer=tokenizer,
)

#Train
trainer.train()
wandb.finish()

#Save model
trainer.save_model("./gpt2-finetuned")
tokenizer.save_pretrained("./gpt2-finetuned")


4.51.3
transformers.training_args
Index(['id', 'annotations', 'drafts', 'predictions', 'data', 'meta',
       'created_at', 'updated_at', 'inner_id', 'total_annotations',
       'cancelled_annotations', 'total_predictions', 'comment_count',
       'unresolved_comment_count', 'last_comment_updated_at', 'project',
       'updated_by', 'comment_authors'],
      dtype='object')


<ipython-input-9-6930487daae7>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_edit['data'] = df_edit['data'].apply(lambda x: x['text'])
<ipython-input-9-6930487daae7>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_edit['file_path'] = df_edit['data'].apply(lambda x: server_prefix+re.sub(f"^{re.escape(prefix)}", "", x))


KeyboardInterrupt: 

#### Evaluation part

In [ ]:

!pip install bs4
import re
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, log_loss
import os
import random
from collections import Counter
from bs4 import BeautifulSoup
import time
device = torch.device('cuda:0')
def extract_plain_text(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()


def extract_all_entities(text):
    pattern = r"<ne type\s*=\s*([^>]+)>(.*?)</ne>"
    return re.findall(pattern, text, re.DOTALL)

def compute_preservation_score(reference, prediction):
    ref_clean = extract_plain_text(reference).split()
    pred_clean = extract_plain_text(prediction).split()
    matched = sum(r == p for r, p in zip(ref_clean, pred_clean))
    return matched / max(len(ref_clean), len(pred_clean)) if max(len(ref_clean), len(pred_clean)) > 0 else 0.0

def compute_ner_metrics_all(reference, prediction):
    ref_entities = extract_all_entities(reference)
    pred_entities = extract_all_entities(prediction)

    ref_counter = Counter(ref_entities)
    pred_counter = Counter(pred_entities)

    y_true_vals = []
    y_pred_probs = []

    all_entities = set(ref_counter.keys()).union(pred_counter.keys())

    for entity in all_entities:
        ref_count = ref_counter[entity]
        pred_count = pred_counter[entity]

        min_count = min(ref_count, pred_count)
        extra_ref = ref_count - min_count
        extra_pred = pred_count - min_count

        #TP
        y_true_vals.extend([1] * min_count)
        y_pred_probs.extend([random.uniform(0.85, 1.0)] * min_count)

        #FN
        y_true_vals.extend([1] * extra_ref)
        y_pred_probs.extend([random.uniform(0.0, 0.3)] * extra_ref)

        #FP
        y_true_vals.extend([0] * extra_pred)
        y_pred_probs.extend([random.uniform(0.7, 1.0)] * extra_pred)

    if not y_true_vals:
        return {
            "f1": 0.0,
            "precision": 0.0,
            "recall": 0.0,
            "accuracy": 0.0,
            "loss": 0.0,
        }

    y_pred_hard = [1 if p > 0.5 else 0 for p in y_pred_probs]

    return {
        "f1": f1_score(y_true_vals, y_pred_hard),
        "precision": precision_score(y_true_vals, y_pred_hard),
        "recall": recall_score(y_true_vals, y_pred_hard),
        "accuracy": accuracy_score(y_true_vals, y_pred_hard),
        "loss": log_loss(y_true_vals, y_pred_probs, labels=[0, 1]),
    }


def evaluate_llm_output(reference: str, prediction: str):
    preservation = compute_preservation_score(reference, prediction)
    ner_metrics = compute_ner_metrics_all(reference, prediction)

    return {
        "preservation_score": preservation,
        "ner_f1_score": ner_metrics["f1"],
        "ner_precision": ner_metrics["precision"],
        "ner_recall": ner_metrics["recall"],
        "ner_accuracy": ner_metrics["accuracy"],
        "eval_loss": ner_metrics["loss"],
    }


def evaluate_batch(references, predictions):
    assert len(references) == len(predictions), "The lists must have the same length"

    results = []
    for ref, pred in zip(references, predictions):
        result = evaluate_llm_output(ref, pred)
        results.append(result)

    def avg(metric):
        return sum(r[metric] for r in results) / len(results)

    return {
        "average_preservation_score": avg("preservation_score"),
        "average_ner_f1_score": avg("ner_f1_score"),
        "average_ner_precision": avg("ner_precision"),
        "average_ner_recall": avg("ner_recall"),
        "average_ner_accuracy": avg("ner_accuracy"),
        "average_eval_loss": avg("eval_loss"),
        "detailed_scores": results
    }

def process_line(line):
  input_text = line
  inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=64)
  inputs = {key: value.to(device) for key, value in inputs.items()}
  outputs = model.generate(
      **inputs,
      num_return_sequences=1,
      do_sample=False,
      use_cache=True,
      num_beams=1
  )
  return tokenizer.decode(outputs[0], skip_special_tokens=True)

references = df_anot1_test['annotated_text'].tolist()
s = time.time()
df_anot1_test['outputs'] = df_anot1_test['file_contents'].apply(lambda x: process_line(x))
references = df_anot1_test['annotated_text'].tolist()
predictions = df_anot1_test['outputs'].tolist()
print(predictions)
uncommon_delimiter = "|||@@@|||"
with open("outputpred.txt", "w") as f:
    for pred in predictions:
        f.write(str(pred))
        f.write(uncommon_delimiter)
with open("outputref.txt", "w") as f:
    for ref in references:
        f.write(str(ref))
        f.write(uncommon_delimiter)
result = evaluate_batch(references, predictions)
e = time.time()
print("Preservation score:", result["average_preservation_score"])
print("F1 skóre (eval_f1)):", result["average_ner_f1_score"])
print("Presnosť (eval_precision):", result["average_ner_precision"])
print("Úplnosť (eval_recall):", result["average_ner_recall"])
print("Presnosť tokenov (eval_accuracy):", result["average_ner_accuracy"])
print("Strata (eval_loss):", result["average_eval_loss"])
print(f"time: {e-s}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

['000000000000e00O000000\nČESKÁ BANKA, filiálka v Turnově,\npřijímá vklady na účty a knížky a vyplácí je neobmezeně v každé výši.\nFinancuje podniky', 'c, : Rozdíl:\n0.850\nna poli ve výšce 5\n— 0.43\n1.28\nv lese\n„\n„\n— 1.06°\nv korunách\nRozdíl: + 0.33\n„\nNa větším díle stanic u', 'Členové výboru.\n(Na základě volby dne 21. dubna 1895 konané.)e otištěno\nprotože vyznačuje', 'K insertu pana dra Šamánka\nv Liberci, v dnešním čísle uveřejněnému,\nsdělujeme, že pan dr. Šamánek ordinuje\nv domě č. 6, Rollgasse, vedle šk', 'Obytné čtvrti se mohou zastavovat podle těchto systémů zastavování (vo-\ndorovného):\n1. volného (otevřeného); k tomu se počítá zastavění domy isolovan�', 'Účet. pošt. spoř. č. 5284..\nOutput: <ne type="obj_a>"<ne type="misc>"čet', 'Zprávy obchodní a reklamní.\nNejdokonalejší čisticí mlýnky a fukary\ndodává firma Adolf Exner v Brandýse n. Orlicí\n(Č Horákových.)\nOutput:právy obchodní a re', 'Prof. dr. Nevěřil: Prameny a literatura k dějinám cyrilo-\nmetodějským; kri

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.p

Preservation score: 0.1384493280294216
F1 skóre (eval_f1)): 0.0
Presnosť (eval_precision): 0.0
Úplnosť (eval_recall): 0.0
Presnosť tokenov (eval_accuracy): 0.0
Strata (eval_loss): 2.32761527434727
time: 32.22649383544922


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.p

In [ ]:

from google.colab import files
!zip -r gpp.zip /content/gpt2-finetuned
files.download('/content/gpp.zip')  # Replace with your actual filename


  adding: content/gpt2-finetuned/ (stored 0%)
  adding: content/gpt2-finetuned/model.safetensors (deflated 9%)
  adding: content/gpt2-finetuned/generation_config.json (deflated 29%)
  adding: content/gpt2-finetuned/tokenizer.model (deflated 55%)
  adding: content/gpt2-finetuned/tokenizer_config.json (deflated 68%)
  adding: content/gpt2-finetuned/config.json (deflated 48%)
  adding: content/gpt2-finetuned/tokenizer.json (deflated 85%)
  adding: content/gpt2-finetuned/special_tokens_map.json (deflated 73%)
  adding: content/gpt2-finetuned/training_args.bin (deflated 51%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
'''
import transformers
model = transformers.GPT2LMHeadModel.from_pretrained("./gpt2-finetuned")
tokenizer = transformers.GPT2Tokenizer.from_pretrained("./gpt2-finetuned")
print("loat")
prompt = "Input: Barack Obama se narodil na Hawaii.\nOutput:"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
outputs = model.generate(input_ids, max_new_tokens=50)
print(tokenizer.decode(outputs[0]))
'''